In [1]:
# Advent Puzzle Day 3
import numpy as np
import os

In [2]:
# helper function to correctly point Justin's python to the correct platform (windows vs osX)
def justin_chdir():
    from sys import platform

    # windows: be sure so to change the <file> location
    if platform == 'win32': os.chdir(r'C:\Users\justi\github\advent_of_code\day 4')

    # Mac OSX: be sure so to change the <file> location
    elif platform == 'darwin': os.chdir('/Users/justinehly/GitHub/advent_of_code/day 4/')

justin_chdir()

In [169]:
input = 'day_4_input.txt'
example = 'day_4_example.txt'
with open(input) as f:
    lines = f.readlines()

with open(example) as f:
    lines2 = f.readlines()

In [184]:
def _init_(lines):
    # Extract the bingo drawings from the dataset
    draw = [lines[0][:-1]]
    draw = draw[0].split(',')
    # set up the bingo boards using a 3d array that is 100 cards of 5x5
    boards = lines[1:]
    boards = [i[:-1] for i in boards if i != '\n']
    squid = []
    for i,x in enumerate(boards):
        squid.append(x.split())
    bingo = []
    b = 0
    for i in range(0, len(squid), 5):
        bingo.append(squid[i:i+5])
    bingo = np.reshape(bingo, (len(bingo),5,5))
    score = np.zeros((len(bingo),5,5))  # create 3D array of zeros to keep track of matched numbers
    return draw, bingo, score



In [185]:
def play_bingo(lines):
    DRAW, BINGO, BINGO2 = _init_(lines)
    test = True  # for the while loop
    winning_card = []  # keep track of winning cards
    winning_idx = [] # keep track of the winning cards binary array from the np.zeros above
    draw_iterator = 0  # iterate through the number drawings
    draws = []  # keep track of what numbers were drawn to the point that a winner was found
    while test:
        marker = np.where(BINGO == DRAW[draw_iterator])  # match the card numbers to the draw number
        draws.append(DRAW[draw_iterator]) #keep track of the drawn numbers
        BINGO2[marker] = 1  
        draw_iterator += 1
        for b, card in enumerate(BINGO2):   # iterate through the 100 cards that are marked as 1 or 0
            for _,row in enumerate(card):  # iterate through each row of the binary cards
                row_set = set(row)  # use a set to determine if a row is all 0s or all 1s
                if len(row_set) == 1:  # we only want rows that are all 1s (matched numbers)
                    iterator = iter(row_set) 
                    if next(iterator) == 1:   # this is how you get numbers from a set
                        winning_card.append(BINGO[b]) # save the winning card
                        winning_idx.append(BINGO2[b]) # save the winning binary card
                        break # get out of the loop
            for _,col in enumerate(card.T):  # transpose the card so that the columns become rows
                col_set = set(col)
                if len(col_set) == 1:
                    iterator = iter(col_set)
                    if next(iterator) == 1:
                        winning_card.append(BINGO[b])
                        winning_idx.append(BINGO2[b])
                        break    
        if len(winning_card) > 0: # if greater than 0 then we hve a winning card or cards
            test = False
            break
    last_draw = DRAW[draw_iterator-1]
    return winning_card, winning_idx, last_draw, draws, BINGO, BINGO2, b



In [186]:
# Calculate the score of the winning card
def calculate_card(card, idx, last_draw):
    last_draw = int(last_draw) # convert the last number draw to an integer
    marker = np.where(idx[0] == 0.) #find the non-matching numbers using the binary card as a map
    
    unmarked = card[0][marker]
    score = sum(unmarked.astype('int')) * last_draw # get the score
    return score
    
    


In [188]:

winner = play_bingo(lines2)
draws_sample = winner[3]
winner = winner[0:3]
calculate_card(*winner)

4512

# Let the squid win

In [200]:
import copy
def squid_wins(lines):
    DRAW, BINGO, BINGO2 = _init_(lines)
    test = True  # for the while loop
    winning_card = []  # keep track of winning cards
    winning_idx = [] # keep track of the winning cards binary array from the np.zeros above
    winning_card_num = []  # iterate through the number drawings
    draws = []  # keep track of what numbers were drawn to the point that a winner was found
    winning_draw = []
    for draw in DRAW:
        marker = np.where(BINGO == draw)  # match the card numbers to the draw number
        BINGO2[marker] = 1  
        b = 0
        for b, card in enumerate(BINGO2):   # iterate through the 100 cards that are marked as 1 or 0
    
            for _,row in enumerate(card):  # iterate through each row of the binary cards
                row_set = set(row)  # use a set to determine if a row is all 0s or all 1s
                if (len(row_set) == 1) and (b not in winning_card_num):  # we only want rows that are all 1s (matched numbers)
                    iterator = iter(row_set) 
                    if next(iterator) == 1:   # this is how you get numbers from a set
                        winning_card.append(copy.deepcopy(BINGO[b])) # save the winning card
                        winning_idx.append(copy.deepcopy(BINGO2[b])) # save the winning binary card
                        winning_draw.append(draw)
                        winning_card_num.append(b)
                        
                        
            for _,col in enumerate(card.T):  # transpose the card so that the columns become rows
                col_set = set(col)
                if (len(col_set) == 1) and (b not in winning_card_num):
                    iterator = iter(col_set)
                    if next(iterator) == 1:
                        winning_card.append(copy.deepcopy(BINGO[b]))
                        winning_idx.append(copy.deepcopy(BINGO2[b]))
                        winning_draw.append(draw)
                        winning_card_num.append(b)
                        
            if b < len(BINGO2):
                b += 1                
            else:
                break
                        
    return winning_card, winning_idx, winning_draw

In [209]:
# Calculate the score of the winning card
def calculate_squid(card, idx, last_draw):
    last_draw = int(last_draw) # convert the last number draw to an integer
    marker = np.where(idx == 0.) #find the non-matching numbers using the binary card as a map
    
    unmarked = card[marker]
    score = sum(unmarked.astype('int')) * last_draw # get the score
    return score

In [211]:
def play_squid(lines):
    squid = squid_wins(lines)
    length = len(squid[0])-1
    winner = squid[0][length], squid[1][length], squid[2][length]
    score = calculate_squid(*winner)
    return score

In [212]:
play_squid(lines2)


1924

In [213]:
play_squid(lines) 

6256